In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Nov 20 03:50:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   28C    P0    43W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!pip install transformers
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.7 MB/s 
     |████████████████████████████████| 7.6 MB 82.3 MB/s 
     |████████████████████████████████| 182 kB 58.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.4 MB/s 
     |████████████████████████████████| 1.3 MB 9.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=ae8de4f669782cfbdf13e362152db40ab848ce3b28ef6d07896d4c7690bb84a5
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_reviews=pd.read_csv('/content/drive/MyDrive/ML_FinalProject/Data/CSV/PA_Reviews.csv', usecols=['business_id', 'text'])

In [ ]:
df_reviews.shape

(1100250, 2)

In [ ]:
from sentence_transformers import SentenceTransformer, LoggingHandler
import logging
import pickle

In [ ]:
#Important, you need to shield your code with if __name__. Otherwise, CUDA runs into issues when spawning new processes.
if __name__ == '__main__':

    #Define the model
    model = SentenceTransformer('all-MiniLM-L6-v2')

    #Start the multi-process pool on all available CUDA devices
    pool = model.start_multi_process_pool()

    #Compute the embeddings using the multi-process pool
    emb = model.encode_multi_process(df_reviews['text'].tolist(), pool)

    # print("Embeddings computed. Shape:", emb.shape) 
    df_out_raw = pd.concat([df_reviews, pd.DataFrame(emb, columns=['sbert_'+str(i) for i in range(len(emb[0]))])], axis = 1)
    #df_out = df_out_raw.groupby('business_id').mean()
    #Optional: Stop the proccesses in the pool
    model.stop_multi_process_pool(pool)

    
    with open('/content/drive/MyDrive/ML_FinalProject/Data/reviews_embeded', 'wb') as f: 
        pickle.dump(df_out_raw, f) 


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
df_out_raw.tail()

,business_id,text,sbert_0,sbert_1,sbert_2,sbert_3,sbert_4,sbert_5,sbert_6,sbert_7,...,sbert_374,sbert_375,sbert_376,sbert_377,sbert_378,sbert_379,sbert_380,sbert_381,sbert_382,sbert_383
1100245,WnT9NIzQgLlILjPT0kEcsQ,Ordered delivery for some tacos on a Saturday ...,-0.025005,0.024450,0.044818,0.087270,-0.115576,-0.071141,-0.061924,-0.082680,...,0.016940,-0.065701,0.004020,-0.008640,0.076991,0.000919,-0.033966,-0.012754,-0.106111,-0.029843
1100246,WnT9NIzQgLlILjPT0kEcsQ,First time trying this restaurant and I had a ...,-0.007562,-0.038003,0.041479,0.075678,-0.113806,-0.016591,-0.019710,-0.050454,...,0.034557,0.003730,0.028163,-0.056047,0.102815,-0.023548,-0.040735,0.026622,-0.095524,0.021599
1100247,WnT9NIzQgLlILjPT0kEcsQ,This restaurant is truly amazing!!! The owner ...,0.070504,0.003356,0.066327,0.024163,-0.122027,-0.019396,-0.001159,-0.076462,...,0.071007,0.026167,0.036363,-0.013390,0.023023,0.037425,0.050846,0.054183,-0.085146,0.036686
1100248,WnT9NIzQgLlILjPT0kEcsQ,Recently got take out from adelita; they were ...,-0.046746,-0.028249,0.016662,0.085571,-0.141710,-0.054469,-0.057059,-0.046827,...,-0.055661,-0.006195,-0.000699,0.038651,-0.016528,0.019300,0.003551,0.008147,-0.102213,0.013176
1100249,WnT9NIzQgLlILjPT0kEcsQ,Another pretty good Mexican place to add into ...,0.091701,-0.029160,0.083100,0.040228,-0.102824,-0.056614,-0.019163,-0.069339,...,0.013631,-0.041331,0.026873,0.020095,0.054239,-0.029445,-0.025692,0.020003,-0.062880,0.026343
